In [1]:
import pandas
import torch
import random as rd
import numpy

rd.seed(0)
numpy.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic=True

In [2]:
train=pandas.read_csv(r"C:\Users\egor-\OneDrive\Рабочий стол\Егор\Mnist_kaggle\train.csv")
print(train.columns)

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)


In [3]:
features=numpy.array(train.drop(columns=["label"]))/255.0
target=numpy.array(train["label"])

In [4]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(features,target,test_size=0.4,shuffle=True)

x_train=x_train.reshape([x_train.shape[0],28,28])
x_test=x_test.reshape([x_test.shape[0],28,28])
x_test[0].shape,x_train[0].shape


((28, 28), (28, 28))

In [5]:
import matplotlib.pyplot as plt
plt.imshow(x_train[0])
plt.show()

<Figure size 640x480 with 1 Axes>

In [6]:
x_train=torch.FloatTensor(x_train)
x_test=torch.FloatTensor(x_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

x_train=x_train.unsqueeze(1).float()
x_test=x_test.unsqueeze(1).float()
x_train.shape,x_test.shape

(torch.Size([25200, 1, 28, 28]), torch.Size([16800, 1, 28, 28]))

In [7]:
class Le_Net(torch.nn.Module):
    def __init__(self,conv_size=5,activation="tanh",pooling="avg",use_batch_normalize=False):
        super(Le_Net,self).__init__()
        self.conv_size=conv_size
        self.use_batch_normalize=use_batch_normalize
        
        if activation=="tanh":
            activation_func=torch.nn.Tanh()
        elif activation=="relu":
            activation_func=torch.nn.ReLU()
        else:
            raise NotImplementedError
        
        if pooling=="avg":
            pooling_layer=torch.nn.AvgPool2d(kernel_size=2,stride=2)
        elif pooling=="max":
            pooling_layer=torch.nn.MaxPool2d(kernel_size=2,stride=2)
        else:
            raise NotImplementedError
        
        
        # first convolutional layer
        if self.conv_size==5:
            self.conv1=torch.nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,padding=2)
        elif self.conv_size==3:
            self.conv1_1=torch.nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,padding=1)
            self.conv1_2=torch.nn.Conv2d(in_channels=6,out_channels=6,kernel_size=3,padding=1)
        else:
            raise NotImplementedError
        
        self.act1=activation_func
        self.bn1=torch.nn.BatchNorm2d(num_features=6)
        self.pool1=pooling_layer

        #second convolutional layer
        if self.conv_size==5:
            self.conv2=torch.nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,padding=0)
        elif self.conv_size==3:
            self.conv2_1=torch.nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,padding=0)
            self.conv2_2=torch.nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,padding=0)
        else:
            raise NotImplementedError
        self.act2=activation_func
        self.bn2=torch.nn.BatchNorm2d(num_features=16)
        self.pool2=pooling_layer
        
        #fullconnected layers
        self.fc1=torch.nn.Linear(5*5*16,120)
        self.act3=activation_func
        self.fc2=torch.nn.Linear(120,84)
        self.act4=activation_func
        self.fc3=torch.nn.Linear(84,10)
        
    def forward(self,x):
        if self.conv_size==5:
            x=self.conv1(x)
        elif self.conv_size==3:
            x=self.conv1_1(x)
            x=self.conv1_2(x)
        x=self.act1(x)
        x=self.bn1(x)
        x=self.pool1(x)
        
        if self.conv_size==5:
            x=self.conv2(x)
        elif self.conv_size==3:
            x=self.conv2_1(x)
            x=self.conv2_2(x)
        x=self.act2(x)
        x=self.bn2(x)
        x=self.pool2(x)
        
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        
        x=self.fc1(x)
        x=self.act3(x)
        x=self.fc2(x)
        x=self.act4(x)
        x=self.fc3(x)
        return x
neural_net=Le_Net(conv_size=3,activation="relu",pooling="max",use_batch_normalize=True)

In [8]:
loss=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(neural_net.parameters(),lr=0.0001)

In [9]:
batch_size=100
for epoch in range(10000):
    order=numpy.random.permutation(len(x_train))
    for start_index in range(0,len(x_train),batch_size):
        optimizer.zero_grad()
        batch_indexes=order[start_index:start_index+batch_size]
        
        x_batch=x_train[batch_indexes]
        y_batch=y_train[batch_indexes]
        
        predict=neural_net.forward(x_batch)
        
        loss_val=loss(predict,y_batch)
        loss_val.backward()
        
        optimizer.step()
        if epoch%100==0:
            
            test_predict=neural_net.forward(x_test).argmax(dim=1)
            print((test_predict==y_test).float().mean())

tensor(0.0976)
tensor(0.1020)
tensor(0.1065)
tensor(0.1110)
tensor(0.1176)
tensor(0.1247)
tensor(0.1343)
tensor(0.1439)
tensor(0.1555)
tensor(0.1679)
tensor(0.1805)
tensor(0.1946)
tensor(0.2098)
tensor(0.2242)
tensor(0.2404)
tensor(0.2562)
tensor(0.2731)
tensor(0.2885)
tensor(0.3015)
tensor(0.3162)
tensor(0.3296)
tensor(0.3452)
tensor(0.3583)
tensor(0.3721)
tensor(0.3896)
tensor(0.4069)
tensor(0.4241)
tensor(0.4439)
tensor(0.4639)
tensor(0.4841)
tensor(0.5016)
tensor(0.5136)
tensor(0.5233)
tensor(0.5352)
tensor(0.5442)
tensor(0.5525)
tensor(0.5583)
tensor(0.5674)
tensor(0.5749)
tensor(0.5816)
tensor(0.5886)
tensor(0.5917)
tensor(0.5973)
tensor(0.6024)
tensor(0.6077)
tensor(0.6124)
tensor(0.6157)
tensor(0.6177)
tensor(0.6219)
tensor(0.6261)
tensor(0.6312)
tensor(0.6365)
tensor(0.6413)
tensor(0.6438)
tensor(0.6479)
tensor(0.6511)
tensor(0.6554)
tensor(0.6594)
tensor(0.6646)
tensor(0.6682)
tensor(0.6721)
tensor(0.6769)
tensor(0.6812)
tensor(0.6867)
tensor(0.6910)
tensor(0.6929)
tensor(0.6

KeyboardInterrupt: 

In [10]:
import pickle
with open(r"C:\Users\egor-\OneDrive\Рабочий стол\Егор\Mnist_kaggle\neural_net.txt","wb") as file:
    pickle.dump(neural_net,file)

In [48]:
test=pandas.read_csv(r"C:\Users\egor-\OneDrive\Рабочий стол\Егор\Mnist_kaggle\test.csv")
test_data=numpy.array(test)/255.0
test_data.shape

(28000, 784)

In [49]:
print(test.columns)

Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)


In [53]:
test_data=test_data.reshape([test_data.shape[0],28,28])

test_data=torch.FloatTensor(test_data).unsqueeze(1).float()
test_data.shape

torch.Size([28000, 1, 28, 28])

In [55]:
predicts=neural_net.forward(test_data)

In [56]:
import csv

In [2]:

with open(r"C:\Users\egor-\OneDrive\Рабочий стол\Егор\Mnist_kaggle\sample_submission.csv", 'w') as f:
    fieldnames = ['ImageId', 'Label']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    i = 1
    for elem in preds:
        writer.writerow({'ImageId': i, 'Label': elem})
        i += 1

NameError: name 'csv' is not defined